# Fluid Construction Grammars

![syntax](img/syntax.png)

## Construction Grammars

Two basic principles:

### 1) Crossing layers

Traditional Linguistics:
![layers1](img/layers1.png)

Construction Grammars:
![layers2](img/layers2.png)

### 2) Bidirectional

<center>$\huge ``the~cat'' \Leftrightarrow \fbox{???} \Leftrightarrow CAT(definite) $</center>

(but, no claim about how $``the~cat''$ or $CAT(definite)$ are represented -- use whatever theory you want)

## Fluid Construction Grammars

Lots of CGs out there:
 - Berkeley Construction Grammar (BCG, 1990's)
 - Sign-Based Construction Grammar (SBCG, 2000's)
 - Goldbergian/Lakovian construction grammar
 - Cognitive grammar
 - Radical construction grammar
 - Embodied construction grammar (ECG)
 - Fluid construction grammar (FCG)
 
Why focus on this one?  https://en.wikipedia.org/wiki/Construction_grammar

> Fluid construction grammar (FCG) was designed by Luc Steels and his collaborators for doing experiments on the origins and evolution of language. FCG is a fully operational and computationally implemented formalism for construction grammars and proposes a uniform mechanism for parsing and production. Moreover, it has been demonstrated through robotic experiments that FCG grammars can be grounded in embodiment and sensorimotor experiences. FCG integrates many notions from contemporary computational linguistics such as feature structures and unification-based language processing. Constructions are considered bidirectional and hence usable both for parsing and production. Processing is flexible in the sense that it can even cope with partially ungrammatical or incomplete sentences. FCG is called 'fluid' because it acknowledges the premise that language users constantly change and update their grammars. 

https://www.fcg-net.org/fcg-interactive/

## Basic Idea

 - Your workspace is a bunch of Structures.  These are collections of slot-value pairs (exactly like Chunks in other formalisms).
 - Constructions are a bunch of things that act on those Structures, changing them.
 
 - So, you might start with the Structure for sound "the" and a structure for the sound "cat" (or the raw phonemes, or whatever).  Constructions will then act on this to build up something complicated.  One component of that complicated resulting structure will be the high-level representation of the cat (maybe `CAT(definite, obj-1)` or `is_cat(obj-1)` or whatever).  
 - Similarly, you can take the same system and feed in `CAT(definite, obj-1)` and it will generate a whole bunch of stuff, one part of whuch is "the" followed by "cat".
 
 - FCG makes no claims about how to represent things, other than to say there's some sort of slot-value pair thing happening.

## Example

![the](img/the.png)



- left-hand-side: Contribution (the thing that this construction makes)
- right-hand-side, above the line: Production (matching rule when producing)
- right-hand-side, below the line: Comprehension (matching rule when comprehending)

Note: when Producing (or Comprehending), the Comprehension part (or the Production part) is also Contributed

I find the above notation horrible.  So here's my attempt at saying the same thing in a different way:

In [1]:
import fcg
class The(fcg.Construction):
    def produce(the):
        the.meaning.definite='TRUE'
    def comprehend(the):
        the.form = 'the'
    def contribute(the):
        the.referent = the.meaning.object
        the.lex_category = 'ARTICLE'

Here's a similar construction for "cat"

In [2]:
class Cat(fcg.Construction):
    def produce(cat):
        cat.meaning.animal_type='CAT'
    def comprehend(cat):
        cat.form = 'cat'
    def contribute(cat):
        cat.referent = cat.meaning.object
        cat.sem_category = 'ANIMATE'
        cat.lex_category = 'NOUN'
        cat.number = 'SINGULAR'

Now let's do something a bit more complex

![syntax](img/syntax.png)

(new notation here: $\leq$ means "is directly to the left of")

Also, note that we're also creating an entirely new structure here (`?NP-unit`), not just contributing to existing ones.

In [3]:
class NounPhrase(fcg.Construction):
    def produce(article, noun):
        article.referent = noun.referent
        article.lex_category = 'ARTICLE'
        noun.lex_category = 'NOUN'
    def comprehend(article, noun):
        article.lex_category = 'ARTICLE'
        noun.lex_category = 'NOUN'
        article.left_of = noun
        noun.right_of = article
    def contribute(article, noun, phrase):
        article.number = noun.number
        noun.syn_func.head = phrase
        article.syn_func.determiner = noun
        phrase.referent = noun.referent
        phrase.category = 'NP'
        phrase.constituents = [article, noun]
        phrase.agreement = noun.number

Now let's try using ths.  We'll start with Production

In [4]:
world = fcg.FCG()
world.add_construction(Cat)
world.add_construction(The)
world.add_construction(NounPhrase)

s1 = world.make_structure()
s1.meaning.animal_type = 'CAT'
s1.meaning.object = 'obj-1'

s2 = world.make_structure()
s2.meaning.definite = 'TRUE'
s2.meaning.object = 'obj-1'

In [5]:
from IPython.core.display import HTML
HTML(fcg.html_structures(world))

In [6]:
world.step('produce')

HTML(fcg.html_structures(world))

found 2 matches
applying Cat.produce(s0)


In [7]:
world.step('produce')

HTML(fcg.html_structures(world))

found 1 matches
applying The.produce(s1)


In [8]:
world.step('produce')

HTML(fcg.html_structures(world))

found 1 matches
applying NounPhrase.produce(s1,s0)


In [9]:
world.step('produce')

found 0 matches


Now we have a full structure, and can strip off the part that should be sent to the speech system

In [14]:
world = fcg.FCG()
world.add_construction(Cat)
world.add_construction(The)
world.add_construction(NounPhrase)

s1 = world.make_structure()
s1.form = 'the'
s2 = world.make_structure()
s2.form = 'cat'
s1.left_of = s2
s2.right_of = s1

HTML(fcg.html_structures(world))

But, what if we just had that output form and wanted to comprehend it?

In [15]:
world.step('comprehend')

HTML(fcg.html_structures(world))

found 2 matches
applying Cat.comprehend(s1)


In [16]:
world.step('comprehend')

HTML(fcg.html_structures(world))

found 1 matches
applying The.comprehend(s0)


In [17]:
world.step('comprehend')

HTML(fcg.html_structures(world))

found 1 matches
applying NounPhrase.comprehend(s0,s1)


Now we can again strip off the high-level conceptual information

## Notes on the change in syntax

- It's not exactly the same, functionally
  - variable binding ambiguity
  - can have same things in `produce` and `comprehend`
  
- But, I think there are good reasons for that

## Could this be implemented in NEF?

- Most of it, I think so, but there's one big exception that might be a deal-breaker

### Parts that fit well with NEF

- Structures are just Semantic Pointers
- Matching is the same things we use in spa Action Selection
- We are always just **adding** to the semantic pointer, not changing slot values!
    - not sure if this is an actual constraint in FCG, but I don't see any counter-examples
- Don't really even need a winner-take-all selection!  Just apply all rules that can apply in parallel!
    - I can't find any examples of FCG doing this, but seems fine to me
    
### Parts that are somewhat problematic with NEF

- Backtracking of Construction applications
   - It is possible (but rare) to have multiple Constructions that could apply, and you have to pick one, but that leads to a dead-end and you have to go back and try another.  In FCG, this is handled with a typical search tree.  In an NEF application, I think it might be possible to do something like reset to the beginning and inhibit the constructions that were used the first time a bit.
   - But, this is pretty rare, and maybe could be dealt with by adding extra info to the initial state the second time through
   
### Parts that I have no idea with the NEF

- Matching to multiple possible structures
  - In the SPA Action stuff, the matching rules always say exactly which group of neurons the match is coming from
  - but here we have a large set of chunks?
  - Where are these stored?  how?  Can I do a parallel match across all of them, or do I need to do that in series?  How is that controlled?
  - maybe we don't need this?  Maybe constructions can be expressed with a fixed small-ish set of Structures?  Or have some mechanism for swapping in and out which one is being matched on?

## Future Plans

- Implement this in HRRs.
  - parallel matching across all
  - parallel application across all (but as a gradual change to the vectors, to be kinda like real dynamics)
  - find a small example to work with, that maybe doesn't have to do full arbitrary matches that I don't know how to implement.